# Data Preparation

In [ ]:
import os
import math
import re
import shutil

last_valid_sequence = 0

def split_data(train_percentage, part_folder):
    print("Splitting data...")

    if not os.path.exists(training_data_path):
        os.makedirs(training_data_path)

    if not os.path.exists(validation_data_path):
        os.makedirs(validation_data_path)

    prefixes = set()
    for filename in os.listdir(os.path.join(raw_directory_path, part_folder)):
        prefix = re.match(r'^(\d+)_', filename)
        if prefix:
            prefixes.add(prefix.group(1))

    for prefix in prefixes:

        file_count = len([f for f in os.listdir(os.path.join(raw_directory_path, part_folder))
                          if f.startswith(f'{prefix}_') and os.path.isfile(os.path.join(raw_directory_path, part_folder, f))])

        total_sequences = round(file_count / 13) #TODO: Wenn hier eine ungerade Zahl herauskommt, wurden nicht alle Files richtig sortiert.
        train_size = round((file_count / 13) * train_percentage)

        # Copy files according to the split into training and validation data directories.
        for current_index in range(1, total_sequences):
            expected_files = get_expected_files(current_index, part_folder, prefix)

            if current_index < train_size:
                destination_path = training_data_path
            else:
                destination_path = validation_data_path

            for expected_file in expected_files:
                source_path = os.path.join(raw_directory_path, expected_file)
                destination_file = os.path.join(destination_path, os.path.basename(expected_file))

                # Check if the file exists before attempting to move it
                if os.path.exists(source_path):
                    shutil.move(source_path, destination_file)
                #else:
                    #print(f"File not found: {source_path}")

    if not last_valid_sequence == 0:
        print(f"Data split into {train_percentage * 100}% training ({train_size} Sequences) and {100 - train_percentage * 100}% validation ({total_sequences - train_size} Sequences).")

    print(f"{os.path.join(raw_directory_path, part_folder)} Splitting complete!")

def get_expected_files(current_index, part_folder, prefix):
    expected_files = set()

    for i in range(11):
        expected_files.add(f'{prefix}_{current_index}_pose_{i}_thermal.png')

    expected_files.add(f'{prefix}_{current_index}_Parameters.txt')
    expected_files.add(f'{prefix}_{current_index}_GT_pose_0_thermal.png')

    return [os.path.join(part_folder, file) for file in expected_files]

def rename_files(current_index, part_folder, prefix):
    expected_files = get_expected_files(current_index, part_folder, prefix)
    for expected_file in expected_files:
        old_path = os.path.join(raw_directory, expected_file)
        new_index = last_valid_sequence + 1
        new_file = expected_file.replace(f'{prefix}_{current_index}', f'{prefix}_{new_index}')
        new_path = os.path.join(raw_directory, new_file)
        os.rename(old_path, new_path)
    #print(f"Renamed sequences with index {current_index} to {last_valid_sequence + 1}")
    return new_index

def cleanup_data(part_folder):
    print(f"{raw_directory} - Cleaning started...")

    global last_valid_sequence
    invalid_sequences_found = False

    prefixes = set()
    for filename in os.listdir(os.path.join(raw_directory_path, part_folder)):
        prefix = re.match(r'^(\d+)_', filename)
        if prefix:
            prefixes.add(prefix.group(1))

    for prefix in prefixes:
        last_valid_sequence = 0

        file_count = len([f for f in os.listdir(os.path.join(raw_directory_path, part_folder))
                          if f.startswith(f'{prefix}_') and os.path.isfile(os.path.join(raw_directory_path, part_folder, f))])

        # TODO better calculation for sequences where more than 50% of the Files are missing.
        approximated_sequences = math.ceil(file_count / 13)

        for current_index in range(approximated_sequences):
            expected_files = get_expected_files(current_index, part_folder, prefix)
            missing_file = None
            found_files = [file_name for file_name in expected_files if
                           os.path.exists(os.path.join(raw_directory_path, part_folder, file_name))]

            if len(found_files) > 0:
                for file_name in expected_files:
                    full_path = os.path.join(raw_directory_path, part_folder, file_name)
                    if not os.path.exists(full_path):
                        missing_file = file_name
                        break

                if missing_file is not None:
                    invalid_sequences_found = True
                    for delete_file in expected_files:
                        delete_path = os.path.join(raw_directory_path, part_folder, delete_file)
                        if os.path.exists(delete_path):
                            os.remove(delete_path)
                    print(f"Sequence {current_index} (Prefix {prefix}) deleted due to missing file: {missing_file}")
                else:
                    if current_index != 0 and (current_index - last_valid_sequence) > 1:
                        last_valid_sequence = rename_files(current_index, part_folder, prefix)
                    else:
                        last_valid_sequence = current_index

    if not invalid_sequences_found:
        print("No invalid sequences found.")


if __name__ == "__main__":
    #TODO Unzip all the zipped batches into ./raw/data
    raw_directory_path = os.path.join(os.path.abspath(os.path.join(os.path.abspath(__file__), '..', '..', '..')), 'data', 'raw')
    training_data_path = os.path.join(os.path.abspath(os.path.join(os.path.abspath(__file__), '..', '..', '..')), 'data', 'train')
    validation_data_path = os.path.join(os.path.abspath(os.path.join(os.path.abspath(__file__), '..', '..', '..')), 'data', 'test')
    train_percentage = 0.8

    # Process Folders
    for part_folder in ["Part1", "Part2"]:
        raw_directory = os.path.join(raw_directory_path, part_folder)

        # Data processing
        cleanup_data(raw_directory)
        split_data(train_percentage, raw_directory) #TODO: Improving.
        print("##########################################")


# AOS

In [9]:
import numpy as np
import cv2
import os
import math
import glm
import pyaos
import re
import glob

def eul2rotm(theta):
    s_1 = math.sin(theta[0])
    c_1 = math.cos(theta[0])
    s_2 = math.sin(theta[1])
    c_2 = math.cos(theta[1])
    s_3 = math.sin(theta[2])
    c_3 = math.cos(theta[2])
    rotm = np.identity(3)
    rotm[0, 0] = c_1 * c_2
    rotm[0, 1] = c_1 * s_2 * s_3 - s_1 * c_3
    rotm[0, 2] = c_1 * s_2 * c_3 + s_1 * s_3

    rotm[1, 0] = s_1 * c_2
    rotm[1, 1] = s_1 * s_2 * s_3 + c_1 * c_3
    rotm[1, 2] = s_1 * s_2 * c_3 - c_1 * s_3

    rotm[2, 0] = -s_2
    rotm[2, 1] = c_2 * s_3
    rotm[2, 2] = c_2 * c_3

    return rotm


def createviewmateuler(eulerang, camLocation):
    rotationmat = eul2rotm(eulerang)
    translVec = np.reshape((-camLocation @ rotationmat), (3, 1))
    conjoinedmat = (np.append(np.transpose(rotationmat), translVec, axis=1))
    return conjoinedmat


def divide_by_alpha(rimg2):
    a = np.stack((rimg2[:, :, 3], rimg2[:, :, 3], rimg2[:, :, 3]), axis=-1)
    return rimg2[:, :, :3] / a


def pose_to_virtualcamera(vpose):
    vp = glm.mat4(*np.array(vpose).transpose().flatten())
    # vp = vpose.copy()
    ivp = glm.inverse(glm.transpose(vp))
    # ivp = glm.inverse(vpose)
    Posvec = glm.vec3(ivp[3])
    Upvec = glm.vec3(ivp[1])
    FrontVec = glm.vec3(ivp[2])
    lookAt = glm.lookAt(Posvec, Posvec + FrontVec, Upvec)
    cameraviewarr = np.asarray(lookAt)
    # print(cameraviewarr)
    return cameraviewarr

def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


def generate_poses(aos, integral_Path, render_fov):
    ########################## Below we generate the poses for rendering #####################################
    # This is based on how renderer is implemented.

    Numberofimages = 11  # Or just the number of images
    Focal_plane = 0  # Focal plane is set to the ground, so it is zero.

    # ref_loc is the reference location or the poses of the images. The poses are the same for the dataset and therefore only the images have to be replaced.
    ref_loc = [[5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]  # These are the x and y positions of the images. It is of the form [[x_positions],[y_positions]]

    altitude_list = [35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35]  # [Z values which is the height]

    center_index = 5  # this is important, this will be the pose index at which the integration should happen. For example if you have 5 images, lets say you want to integrate all 5 images to the second image position. Then your center_index is 1 as index starts from zero.

    site_poses = []
    for i in range(Numberofimages):
        EastCentered = (ref_loc[0][i] - 0.0)    # Get MeanEast and Set MeanEast
        NorthCentered = (0.0 - ref_loc[1][i])   # Get MeanNorth and Set MeanNorth
        M = createviewmateuler(np.array([0.0, 0.0, 0.0]), np.array([ref_loc[0][i], ref_loc[1][i], - altitude_list[i]]))
        print('m', M)
        ViewMatrix = np.vstack((M, np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float32)))
        print(ViewMatrix)
        camerapose = np.asarray(ViewMatrix.transpose(), dtype=np.float32)
        print(camerapose)
        site_poses.append(camerapose)  # site_poses is a list now containing all the poses of all the images in a certain format that is accepted by the renderer.

    # Read the generated images from the simulator and store in a list

    numbers = re.compile(r'(\d+)')

    imagelist = []

    for img in sorted(glob.glob(os.path.join(training_data_path, '*.png')), key=numericalSort):
        n = cv2.imread(img)
        imagelist.append(n)

    aos.clearViews()  # Every time you call the renderer you should use this line to clear the previous views
    for i in range(len(imagelist)):
        aos.addView(imagelist[i], site_poses[i], "DEM BlobTrack")  # Here we are adding images to the renderer one by one.
    aos.setDEMTransform([0, 0, Focal_plane])

    proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[center_index]), render_fov)
    tmp_RGB = divide_by_alpha(proj_RGBimg)
    cv2.imwrite(os.path.join(integral_Path, 'integral.png'),
                tmp_RGB)  # Final result. Check the integral result in the integrals folder.


def aos_renderer():
    w, h, fovDegrees = 512, 512, 50  # resolution and field of view. This should not be changed.
    render_fov = 50

    if 'window' not in locals() or window == None:
        window = pyaos.PyGlfwWindow(w, h, 'AOS')

    aos = pyaos.PyAOS(w, h, fovDegrees)

    set_folder = os.path.abspath(os.path.join(os.path.abspath(os.path.dirname(__file__)), os.pardir, 'python'))
    aos.loadDEM(os.path.join(set_folder, 'zero_plane.obj'))
    return aos, render_fov


def check_directories():
    try:
        script_path = os.path.abspath(__file__)
    except NameError:
        script_path = None

    if script_path is not None:
        Download_Location = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname(script_path)), 'results'))
    else:
        Download_Location = os.path.abspath(os.path.join(os.getcwd(), '..', 'results'))

    print("Download Location:", Download_Location)

    Integral_Path = os.path.join(Download_Location, 'integrals')

    if not os.path.exists(Integral_Path):
        os.mkdir(Integral_Path)
    else:
        print(f"The directory '{Integral_Path}' already exists.")


if __name__ == "__main__":
    training_data_path = os.path.join(os.path.abspath(os.path.join(os.path.abspath(__file__), '..', '..', '..')), 'data','train')

    integral_path = check_directories()
    aos, render_fov = aos_renderer()
    generate_poses(aos, integral_path, render_fov)

NameError: name '__file__' is not defined

# Model

In [ ]:
import sys
sys.path.append('./models')
sys.path.append('./utils')

import torch

from aos_deeplab import AosDeepLab
from train_deeplab import train_deeplab, check_gpu_availability

model = AosDeepLab()
print(f'GPU available: {check_gpu_availability()}')

trained_model = train_deeplab(model, num_epochs=10)

# torch.save(trained_model.state_dict(), "aosdeeplab_model.pth")

# checkpoints

In [ ]:
import torch
import pickle
import os

def get_checkpoint(model, optimizer, checkpoint_dir = "./checkpoints"):
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        current_index = checkpoint['current_index']
        current_batch = checkpoint['current_batch']
        print(f"Checkpoint loaded. Current index: {current_index}, Current batch: {current_batch}")
        return current_index, current_batch
    else:
        print("No checkpoint found. Training from scratch.")
        return 0, 0
    
def save_checkpoint(model, optimizer, current_index, current_batch, checkpoint_dir='./checkpoints'):
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'current_index': current_index,
        'current_batch': current_batch,
    }, checkpoint_path)

    data = {"current_index": current_index, "current_batch": current_batch}
    with open(os.path.join(checkpoint_dir, 'checkpoint_info.pickle'), 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)